In [17]:
import os
import numpy as np
from math import ceil
from glob import glob
from easydict import EasyDict
from Analysis_template import AWS_s3
from Analysis_template import Model_template

import torch
from torch import nn, optim
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

fit cross validation <- scaler 추가하기  
sampling

In [18]:
aws = AWS_s3(save_json=True)

Enter your aws_access_key_id:········
Enter your aws_secret_access_key:········
접속 성공


10 ~ 16 step을 이용

In [3]:
%time df = aws.load_data('OPEN/samsung_sample/samsung_recipe.csv')

CPU times: user 3.82 s, sys: 871 ms, total: 4.69 s
Wall time: 6.05 s


In [4]:
recipe1 = df[df['recipe'] == 1]
recipe2 = df[df['recipe'] == 2]

recipe1 = np.array(list(map(lambda x : [x[1].values[:,2:-1], x[0][1], len(x[1].values)], 
                            recipe1.groupby(['count', 'defect']))))

recipe2 = np.array(list(map(lambda x : [x[1].values[:,2:-1], x[0][1], len(x[1].values)], 
                            recipe2.groupby(['count', 'defect']))))

recipe1_max_length = np.max(recipe1[:,-1])
recipe2_max_length = np.max(recipe2[:,-1])
max_length = recipe1_max_length if recipe1_max_length > recipe2_max_length else recipe2_max_length

In [5]:
class Recipe_dataset(Dataset):
    def __init__(self, recipe1, recipe2, max_length):
        self.X1 = recipe1[:,0]
        self.y1 = recipe1[:,1]
        self.X2 = recipe2[:,0]
        self.y2 = recipe2[:,1]
        
        self.X1_length = len(self.X1)
        self.X = np.concatenate((self.X1, self.X2))
        self.y = np.concatenate((self.y1, self.y2))
        self.max_length = max_length

    # data에 맞게 전처리 해야함.
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X = np.concatenate((self.X[idx], np.zeros((self.max_length - len(self.X[idx]), 65))), 
                           axis = 0)
        if idx >= self.X1_length:
            return X.astype(np.float32), np.int(self.y[idx]), 2
        else:
            return X.astype(np.float32), np.int(self.y[idx]), 1
        
    def __len__(self):
        return len(self.X)

In [6]:
class Model(Model_template) :
    def __init__(self, hyperperameters) :
        super().__init__(hyperparameters)
        
        self.loss = nn.NLLLoss()
        self.kernel = hyperperameters['encoding_kernel']
        self.hidden = hyperperameters['attn_kernel']
        
        # time-series encoding
        self.conv = nn.Conv2d(1, self.kernel, (65,3), bias = False)
        
        self.activate_1 = True
        # convolution attention 
        self.attn_conv = nn.Conv2d(1, self.hidden, (self.kernel, 1), bias = False) # bias 지우기
        self.dropout = nn.Dropout2d(p=0.5)
        self.fc_recipe1_1 = nn.Linear(self.kernel * self.hidden, 128)
        self.fc_recipe1_2 = nn.Linear(128, 128)
        self.fc_recipe1_3 = nn.Linear(128, 2)
        
        self.fc_recipe2_1 = nn.Linear(self.kernel * self.hidden, 128)
        self.fc_recipe2_2 = nn.Linear(128, 128)
        self.fc_recipe2_3 = nn.Linear(128, 2)
    
    def forward(self, x) :
        batch, time, variable = x.size()
        x = x.view(-1,1,variable,time)
        x = F.tanh(self.conv(x))
        x = torch.transpose(x, 1,2)
        original_x = x
        x = self.attn_conv(x)
        x = F.softmax(x, dim = -1)
        x = self.dropout(x)
        
        x = torch.squeeze(x, dim = 2)
        x = torch.transpose(x, 1,2)
        original_x = torch.squeeze(original_x, dim = 1)
        output = torch.bmm(original_x, x)
        output = torch.flatten(output, start_dim=1)

        self.fc_recipe1_1.weight.requires_grad=True
        self.fc_recipe1_2.weight.requires_grad=True
        self.fc_recipe1_3.weight.requires_grad=True
        self.fc_recipe2_1.weight.requires_grad=True
        self.fc_recipe2_2.weight.requires_grad=True
        self.fc_recipe2_3.weight.requires_grad=True
        
        
        if self.activate_1 :
            ## Recipe1 Network ##
            self.fc_recipe2_1.weight.requires_grad=False
            self.fc_recipe2_2.weight.requires_grad=False
            self.fc_recipe2_3.weight.requires_grad=False
            
            output = F.relu(self.fc_recipe1_1(output))
            output = F.relu(self.fc_recipe1_2(output))
            output = self.fc_recipe1_3(output)
            self.activate_1 = not self.activate_1
        else :
            ## Recipe2 Network ##
            self.fc_recipe1_1.weight.requires_grad=False
            self.fc_recipe1_2.weight.requires_grad=False
            self.fc_recipe1_3.weight.requires_grad=False
            output = F.relu(self.fc_recipe2_1(output))
            output = F.relu(self.fc_recipe2_2(output))
            output = self.fc_recipe2_3(output)
            self.activate_1 = not self.activate_1

        return F.log_softmax(output, dim=1)

    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        scheduler = StepLR(optimizer, step_size=self.hparams.step_size, 
                           gamma=self.hparams.gamma)
        
        return [optimizer], [scheduler]
    
    def train_data_change(self, train_data):
        recipe1 = np.array([(i[0], i[1], i[-1]) for i in train_data if i[-1] == 1])
        recipe2 = np.array([(i[0], i[1], i[-1]) for i in train_data if i[-1] == 2])
        len1 = len(recipe1)
        len2 = len(recipe2)
        
        if len1 > len2:
            max_length = ceil(len1/self.hparams.batch_size) * self.hparams.batch_size
            index2 = np.random.choice(range(len2), max_length)
            index1 = np.random.choice(range(len1), max_length)
            
        else:
            max_length = ceil(len2/self.hparams.batch_size) * self.hparams.batch_size
            index1 = np.random.choice(range(len1), max_length)
            index2 = np.random.choice(range(len2), max_length)
            
        np.random.shuffle(index1)
        np.random.shuffle(index2)
        split_recipe1 = np.split(index1, self.hparams.batch_size)
        split_recipe2 = np.split(index2, self.hparams.batch_size)
        
        final_dataset = []
        for num in range(int(max_length/self.hparams.batch_size)):
            add = recipe1[split_recipe1[num]].tolist()+recipe2[split_recipe2[num]].tolist()
            final_dataset += add
            
        return final_dataset

In [7]:
 hyperparameters = EasyDict({'lr' : 0.0001,
                            'max_epochs' :100,
                            'step_size' : 10, # scheduler, epoch 단위
                            'gamma' : 0.9, # schduler
                            'batch_size' : 64, # train_batch_size
                            'test_batch_size' : -1, # test_batch_size
                            'gpus' : [0],
                            'num_workers' : 16,
                            'auto_lr_find' : False,
                            'save_top_k' : 5,
                            'folder' : 'best_model',
                             'early_stopping' : True,
                             'patience' : 20,
                             'encoding_kernel' : 10, 
                             'attn_kernel' : 10 
                            })


if not os.path.isdir(hyperparameters['folder']) :
    os.mkdir(hyperparameters['folder'])

In [8]:
recipe_dataset = Recipe_dataset(recipe1, recipe2, max_length)

In [9]:
model = Model(hyperparameters)

In [10]:
model.fit_cross_validation(recipe_dataset, 5, 0, train_shuffle = False, scaler='time-series')

EarlyStopping mode auto is unknown, fallback to auto mode.
EarlyStopping mode set to min for monitoring val_loss.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



  | Name         | Type      | Params
-------------------------------------------
0 | loss         | NLLLoss   | 0     
1 | conv         | Conv2d    | 1 K   
2 | attn_conv    | Conv2d    | 100   
3 | dropout      | Dropout2d | 0     
4 | fc_recipe1_1 | Linear    | 12 K  
5 | fc_recipe1_2 | Linear    | 16 K  
6 | fc_recipe1_3 | Linear    | 258   
7 | fc_recipe2_1 | Linear    | 12 K  
8 | fc_recipe2_2 | Linear    | 16 K  
9 | fc_recipe2_3 | Linear    | 258   


epoch : 0, training loss : 0.6488, validation loss : 0.5254, ckpt_path = 200911_22:02:22/1_fold/epoch=0_val_loss=0.5254


epoch : 1, training loss : 0.4879, validation loss : 0.4979, ckpt_path = 200911_22:02:22/1_fold/epoch=1_val_loss=0.4979


epoch : 2, training loss : 0.3475, validation loss : 0.2980, ckpt_path = 200911_22:02:22/1_fold/epoch=2_val_loss=0.2980


...

epoch : 52, training loss : 0.1116, validation loss : 0.0759, ckpt_path = 200911_22:02:22/5_fold/epoch=52_val_loss=0.0759


epoch : 53, training loss : 0.1151, validation loss : 0.0784, ckpt_path = 200911_22:02:22/5_fold/epoch=53_val_loss=0.0784


epoch : 54, training loss : 0.1070, validation loss : 0.0767, ckpt_path = 200911_22:02:22/5_fold/epoch=54_val_loss=0.0767


Saving latest checkpoint..
Epoch 00056: early stopping triggered.


epoch : 55, training loss : 0.0938, validation loss : 0.0778, ckpt_path = 200911_22:02:22/5_fold/epoch=55_val_loss=0.0778



In [11]:
def accuracy(y_hat, y): # manual metric example
    return torch.sum(torch.max(y_hat, axis = 1)[1] == y).item()/len(y_hat)

In [12]:
model.test_cross_validation(recipe_dataset, 'a', '200911_02:28:56', accuracy)

In [13]:
# model = model.load_model('200911_00:13:32')

# def accuracy(y_hat, y): # manual metric example
#     return torch.sum(torch.max(y_hat, axis = 1)[1] == y).item()/len(y_hat)

# model.test(recipe1_dataset, 'accuracy', accuracy)